In [1]:
import numpy as np 
import copy
import re
import sys
import json

fp = open("hmm-training-data/it_isdt_train_tagged.txt","r", encoding='utf8')
#filename = sys.argv[1]
#fp = open(filename,"r", encoding='utf8')
lines = fp.readlines()
transitions = {}
emisions = {}
uniqueness = {}

i = 0
for line in lines:
    line_list = line.strip().split()
    for word in line_list:
        pos = word.split('/')[-1]
        n = len(word) - len(pos) - 1
        emision = word[:n]
        
        if(pos in transitions.keys()):
            transitions[pos] += 1
        else:
            transitions[pos] = 1
        
        if(emision in emisions.keys()):
            emisions[emision] += 1
        else:
            emisions[emision] = 1
            
        if(pos in uniqueness.keys()):
            uniqueness[pos].add(emision)
        else:
            uniqueness[pos] = set(emision)

#print("The tags are: ", transitions, "Count: ", len(transitions))
#print("The words are: ", emisions, "Count: ", len(emisions))

uniqueness = {key: len(value) for key, value in uniqueness.items()}
total = sum(uniqueness.values())
uniqueness = {key: value/total for key, value in uniqueness.items()}
#uniqueness

In [2]:
transition_matrix = {}
blank = {k: 0 for k in transitions.keys()}
for i in transitions.keys():
    transition_matrix[i] = copy.deepcopy(blank)
emision_matrix = {}
for i in emisions.keys():
    emision_matrix[i] = copy.deepcopy(blank)

In [75]:
for line in lines:
    line_list = line.split()
    prev_tag = ""
    for i in range(len(line_list)):
        word = line_list[i]
        pos = word.split('/')[-1]
        n = len(word) - len(pos) - 1
        emision = word[:n]
        emision_matrix[emision][pos] += 1
        if(i==0):
            prev_tag = pos
            continue

        transition_matrix[prev_tag][pos] += 1
        prev_tag = pos

emision_matrix["__________"] = uniqueness
#transition_matrix
#emision_matrix

In [76]:
alpha = 0.0001
for i in emision_matrix.keys():
    emision_matrix[i] = {key: value+alpha for key, value in emision_matrix[i].items()}
    total = sum(emision_matrix[i].values())
    emision_matrix[i] = {key: value/total for key, value in emision_matrix[i].items()}
    #print(sum(emision_matrix[i].values()))

for i in transition_matrix.keys():
    transition_matrix[i] = {key: value+alpha for key, value in transition_matrix[i].items()}
    total = sum(transition_matrix[i].values())
    transition_matrix[i] = {key: value/total for key, value in transition_matrix[i].items()}
    #print(sum(transition_matrix[i].values()))

#print(emision_matrix.keys())
#print(transition_matrix.keys())

In [77]:
new_dict = {"e":emision_matrix, "t":transition_matrix}
with open("hmmmodel.txt","w") as file:
    file.write(json.dumps(new_dict)) 